## Setup

In [1]:
import boa, os
from snekmate.tokens import ERC20
from dotenv import load_dotenv

%reload_ext boa.ipython

# Constants
load_dotenv(".env")
RPC_URL = f"https://arb-mainnet.g.alchemy.com/v2/{os.getenv('ARB_ALCHEMY_KEY')}"
ARBISCAN_KEY = os.getenv("ARBISCAN_KEY")
ARBISCAN_API = "https://api.arbiscan.io/api"
FACTORY_ADDR = "0xcaec110c784c9df37240a8ce096d352a75922dea"

In [2]:
# Fork an RPC Node
boa.env.fork(RPC_URL)
print(f"Current block: {boa.env.evm.patch.block_number}")

Current block: 213969478


In [3]:
# Redeploy collateral token from lesson 1
collateral_token = ERC20.deploy("Vyper wif hat", "VWIF", 100_000_000, "", "")
print(
    f"Deployed to {collateral_token.address} "
    + f"and minted\n{collateral_token.balanceOf(collateral_token.owner()) / 10**18:,.0f} "
    + f"${collateral_token.symbol()} "
    + f"tokens to token owner {collateral_token.owner()}"
)

Deployed to 0x0880cf17Bd263d3d3a5c09D2D86cCecA3CcbD97c and minted
100,000,000 $VWIF tokens to token owner 0x00dE89C733555886f785b0C32b498300297e481F


## Load Llama Lend Factory

In [4]:
# Load Factory Contract
factory = boa.from_etherscan(FACTORY_ADDR, "Factory", ARBISCAN_API, ARBISCAN_KEY)
print(f"Currently there are {factory.market_count()} Llama Lend markets on Arbitrum")

Currently there are 9 Llama Lend markets on Arbitrum


## Deploy Oracle
*May need to run `jupyter lab extension enable boa` or `jupyter lab extension enable vyper`*

*Contract from: [github/@curvefi/curve-stablecoin](https://github.com/curvefi/curve-stablecoin/blob/master/contracts/testing/DummyPriceOracle.vy)*

In [5]:
%%vyper dummy_oracle

price: public(uint256)
ADMIN: immutable(address)

@external
def __init__(admin: address, price: uint256):
    self.price = price
    ADMIN = admin

@external
def price_w() -> uint256:
    # State-changing price oracle in case we want to include EMA
    return self.price

@external
def set_price(price: uint256):
    assert msg.sender == ADMIN
    self.price = price

In [6]:
# Deploy Dummy Oracle
oracle = dummy_oracle.deploy(boa.env.eoa, 10**18)
print(
    f"Dummy oracle deployed to {oracle.address} with oracle price of {oracle.price_w() / 10**18}"
)

Dummy oracle deployed to 0x2cb6bCe32aeF4eD506382896e702DE7Ff109D9E9 with oracle price of 1.0


## Launch Pool

In [7]:
# Deploy
vault_addr = factory.create(
    factory.STABLECOIN(),
    collateral_token,
    1000,
    500000000000000,
    13000000000000000,
    10000000000000000,
    oracle,
    "VyperWif",
    0,
    4756468797,
)
print(
    f"Deployed to {vault_addr}\nThere are now {factory.market_count()} markets on Llama Lend"
)

Deployed to 0xd3cA9BEc3e681b0f578FD87f20eBCf2B7e0bb739
There are now 10 markets on Llama Lend
